# Preperations

In [ ]:
%cd Mini\ Project\ 4

/content/Mini Project 4


In [ ]:
! git clone https://github.com/zhengxxn/adaptive-knn-mt.git
! ls

Cloning into 'adaptive-knn-mt'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (808/808), done.
remote: Compressing objects: 100% (648/648), done.
remote: Total 808 (delta 139), reused 803 (delta 134), pack-reused 0
Receiving objects: 100% (808/808), 3.99 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (139/139), done.
adaptive-knn-mt  Model


In [ ]:
! pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu102.html

Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 8.0 MB 5.3 MB/s 


In [ ]:
! pip install sacremoses
! pip install sacrebleu==1.4.12

     |████████████████████████████████| 895 kB 5.4 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 17.1 MB 71 kB/s 


In [ ]:
! pip install -e adaptive-knn-mt/
! pip install tensorboardX

Obtaining file:///content/Mini%20Project%204/adaptive-knn-mt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 145 kB 5.2 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
     |████████████████████████████████| 112 kB 64.7 MB/s 
     |████████████████████████████████| 596 kB 42.2 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=460da37402dd679c7a43949afdd8679456d0db741b3aa9ff81bb8595a459d4fb
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for fairseq


In [ ]:
! pwd
%cd ..

/content/Mini Project 4
/content


In [ ]:
! unzip it.zip
! unzip koran.zip
! unzip law.zip
! unzip medical.zip

Archive:  it.zip
   creating: it/
  inflating: it/train.de-en.en.idx   
   creating: __MACOSX/
   creating: __MACOSX/it/
  inflating: __MACOSX/it/._train.de-en.en.idx  
  inflating: it/valid.de-en.de.bin   
  inflating: __MACOSX/it/._valid.de-en.de.bin  
  inflating: it/dict.de.txt          
  inflating: __MACOSX/it/._dict.de.txt  
  inflating: it/train.de-en.en.bin   
  inflating: __MACOSX/it/._train.de-en.en.bin  
  inflating: it/valid.de-en.de.idx   
  inflating: __MACOSX/it/._valid.de-en.de.idx  
  inflating: it/test.de-en.de.bin    
  inflating: __MACOSX/it/._test.de-en.de.bin  
  inflating: it/test.de-en.de.idx    
  inflating: __MACOSX/it/._test.de-en.de.idx  
  inflating: it/test.de-en.en.idx    
  inflating: __MACOSX/it/._test.de-en.en.idx  
  inflating: it/test.de-en.en.bin    
  inflating: __MACOSX/it/._test.de-en.en.bin  
  inflating: it/dict.en.txt          
  inflating: __MACOSX/it/._dict.en.txt  
  inflating: it/train.de-en.de.bin   
  inflating: __MACOSX/it/._train.de-e

In [ ]:
! pip install faiss-gpu

  Using cached faiss_gpu-1.7.1.post3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (90.1 MB)


# The Koran Dataset

## Create Datastore

In [ ]:
%env DSTORE_SIZE=524400
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=koran
%env DATASTORE_PATH=data_store
%env PROJECT_PATH=adaptive-knn-mt

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/save_datastore.py $DATA_PATH \
    --dataset-impl mmap \
    --task translation \
    --valid-subset train \
    --path $MODEL_PATH \
    --max-tokens 4096 --skip-invalid-size-inputs-valid-test \
    --decoder-embed-dim 1024 --dstore-fp16 --dstore-size $DSTORE_SIZE --dstore-mmap $DATASTORE_PATH
 
# 4096 and 1024 depend on your device and model separately

env: DSTORE_SIZE=524400
env: MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
env: DATA_PATH=koran
env: DATASTORE_PATH=data_store
env: PROJECT_PATH=adaptive-knn-mt
2021-12-13 00:57:15 | INFO | fairseq_cli.validate | loading model(s) from Model/wmt19.de-en.ffn8192.pt
Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, avg_k=False, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='koran', dataset_impl='mmap', ddp_backend='c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=1024, device_id=0, distributed_backend='nccl', distributed_init_method='tcp://learnfair0218:1259

## Build Faiss Index

In [ ]:
%env PROJECT_PATH=adaptive-knn-mt
%env DSTORE_PATH=data_store
%env DSTORE_SIZE=524400

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/train_datastore_gpu.py \
    --dstore_mmap $DSTORE_PATH \
    --dstore_size $DSTORE_SIZE \
    --dstore-fp16 \
    --faiss_index ${DSTORE_PATH}/knn_index \
    --ncentroids 4096 \
    --probe 32 \
    --dimension 1024

env: PROJECT_PATH=adaptive-knn-mt
env: DSTORE_PATH=data_store
env: DSTORE_SIZE=524400
Namespace(code_size=64, concat_file_path=None, dimension=1024, dstore_fp16=True, dstore_mmap='data_store', dstore_size=524400, faiss_index='data_store/knn_index', load_multiple_files=False, multiple_files_size=None, multiple_key_files=None, multiple_val_files=None, ncentroids=4096, num_keys_to_add_at_a_time=500000, probe=32, seed=1, starting_point=0)
load dstore fp16 524400 1024
done.
Adding Keys
tcmalloc: large alloc 2048000000 bytes == 0x5610cef08000 @  0x7ff57556c1e7 0x7ff5730ec46e 0x7ff57313cc7b 0x7ff57313cd97 0x7ff5731d5887 0x56106d7654b0 0x56106d856e1d 0x56106d7d8e99 0x56106d7d39ee 0x56106d7d36f3 0x56106d89d4c2 0x56106d89d83d 0x56106d89d6e6 0x56106d875163 0x56106d874e0c 0x7ff574356bf7 0x56106d874cea
Added 1000000 tokens so far
Writing Index 1000000
Adding total 524400 keys
Adding took 25.523330450057983 s
Writing Index
Writing index took 0.6554005146026611 s


## Train Adaptive kNN-MT Model

This part is encapsulated into a shell script, since colab doesn't support shell loops (and rewriting them into native Python code requires too much effort).

In [ ]:
! bash train1.sh

2021-12-13 01:03:25 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer_wmt19_de_en_with_datastore', attention_dropout=0.1, avg_k=False, batch_size=32, batch_size_valid=32, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='koran', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=1024, device_id=0, disa

In [ ]:
! bash train.sh

2021-12-12 13:20:20 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer_wmt19_de_en_with_datastore', attention_dropout=0.1, avg_k=False, batch_size=32, batch_size_valid=32, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='koran', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=1024, device_id=0, disa

## Adaptive *k*NN-MT Inference

In [30]:
%env DSTORE_SIZE=524400
%env MODEL_PATH=model_record/train-hid32-maxk4/checkpoint.best_loss_1.83.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=medical
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

! mkdir -p "$OUTPUT_PATH"

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH \
--gen-subset test --path $MODEL_PATH \
--arch transformer_wmt19_de_en_with_datastore \
--beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
--scoring sacrebleu \
--batch-size 32 \
--tokenizer moses --remove-bpe \
--model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True, \
'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, \
'dstore_fp16': True, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', \
'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True, \
'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
| tee "$OUTPUT_PATH"/generate.txt

!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

Streaming output truncated to the last 5000 lines.
D-1877	-2.473219156265259	From the use of EXUBERA in patients who need dose adjustment in smaller steps.
P-1877	-1.5204 -1.3336 -0.3940 -0.0312 -0.3370 -0.4523 -1.2070 -0.0260 -0.3474 -0.8420 -0.6236 -1.9480 -1.9498 -0.4209 -0.3909 -0.0372 -0.3195 -0.9493 -0.7607 -1.3799 -0.0965
S-1115	- wenn Sie überempfindlich (allergisch) gegen Bevacizumab oder einen der sonstigen
T-1115	- you are allergic (hypersensitive) to bevacizumab or to any of the other ingredients of Avastin.
H-1115	-1.244767189025879	- if you are hypersensitive ( allergic ) to bevacizumab or any of the other
D-1115	-1.244767189025879	- if you are hypersensitive (allergic) to bevacizumab or any of the other
P-1115	-0.5776 -0.4212 -0.0781 -0.1214 -0.4111 -0.1313 -0.9218 -0.3113 -0.4177 -0.0227 -0.0433 -0.5492 -0.2317 -0.3292 -0.1312 -0.4598 -0.0241 -0.2979 -0.7402 -0.1006 -0.3356 -1.0852 -0.4257
S-349	Jede Filmtablette enthält 20 mg Memantinhydrochlorid, entsprechend 16,62 mg

## Base NMT Inference

In [ ]:
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=koran
%env DATASTORE_PATH=data_store
%env PROJECT_PATH=adaptive-knn-mt
%env OUTPUT_PATH=base_output

! mkdir -p $OUTPUT_PATH

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/fairseq_cli/generate.py $DATA_PATH\
    --gen-subset test \
    --path $MODEL_PATH \
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
    --scoring sacrebleu \
    --max-tokens 4096 \
    --tokenizer moses --remove-bpe | tee $OUTPUT_PATH/generate.txt

! grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
! grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
! grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
! grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

Streaming output truncated to the last 5000 lines.
D-1926	-2.6042535305023193	Glory be to Him! He is God, the One, the Overpowering.
P-1926	-1.1789 -1.0888 -1.5079 -0.3125 -0.1500 -0.3545 -0.8273 -0.6081 -0.1687 -0.6574 -0.2851 -1.1868 -1.4168 -0.6251 -0.4257 -2.8988 -1.4622 -0.1315 -0.2571 -0.1713
S-1941	Allah wird ganz gewiß diejenigen kennen, die die Wahrheit sprechen, und Er wird ganz gewiß die Lügner kennen.
T-1941	So Allah will surely know those who are true and He will surely know the liars.
H-1941	-1.6841493844985962	Allah will certainly know those who speak the truth , and He will certainly know the liars .
D-1941	-1.6841493844985962	Allah will certainly know those who speak the truth, and He will certainly know the liars.
P-1941	-1.5386 -0.0620 -0.7162 -1.9569 -0.1816 -0.3945 -0.1415 -0.3327 -0.3052 -0.1241 -0.8190 -0.1825 -0.3742 -0.2400 -1.3921 -0.1027 -0.8157 -0.2564 -0.1526 -0.2162 -0.1596
S-1981	Und erfüllt die (eingegangene) Verpflichtung. Gewiß, nach der (Erfüllung der

## Vanilla *k*NN-MT Inference

In [ ]:
%env DSTORE_SIZE=524400
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=koran
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=plain_output

! mkdir -p "$OUTPUT_PATH"

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH \
    --gen-subset test\
    --path $MODEL_PATH --arch transformer_wmt19_de_en_with_datastore \
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
    --scoring sacrebleu \
    --batch-size 32 \
    --tokenizer moses --remove-bpe \
    --model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True, \
    'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, 'dstore_fp16': True, 'k': 4, 'probe': 32, \
    'knn_sim_func': 'do_not_recomp_l2', 'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True, \
    'knn_lambda_type': 'fix', 'knn_lambda_value': 0.7, 'knn_temperature_type': 'fix', 'knn_temperature_value': 10, \
     }" \
    | tee "$OUTPUT_PATH"/generate.txt

! grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
! grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
! grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
! grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

Streaming output truncated to the last 5000 lines.
D-1092	-1.8455586433410645	This is the bounty of Allah, He gives whosoever He please, and Allah is Extremely Munificent.
P-1092	-1.0811 -0.0467 -0.6244 -0.4748 -0.0249 -0.0639 -0.8359 -0.0156 -0.6568 -0.3833 -0.7817 -0.7239 -0.9539 -0.0433 -0.0270 -0.0370 -0.8506 -1.1023 -0.3612 -1.2994 -0.0168 -0.2968 -1.3247 -0.0970 -0.6438 -0.5143 -0.5127 -0.2761 -0.0904 -0.0435
S-811	Nimm Almosen von ihrem Besitz, auf daß du sie dadurch reinigen und läutern mögest.
T-811	Accept the offerings they make from their wealth in order to cleanse and purify them for progress, and invoke blessings upon them.
H-811	-1.5389395952224731	Take charity from their wealth , in order that they are thereby cleansed and purified .
D-811	-1.5389395952224731	Take charity from their wealth, in order that they are thereby cleansed and purified.
P-811	-0.6765 -1.0766 -0.2146 -0.1242 -0.3329 -0.2529 -1.6463 -0.2127 -0.7508 -0.9372 -0.9400 -0.8180 -0.2640 -0.8623 -0.0398 -0.